In [1]:
%cd /private/home/pakamienny/Research/symbolicregression
%load_ext autoreload
%autoreload 2


/private/home/pakamienny/Research/symbolicregression


In [2]:
import numpy as np


from symbolicregression.envs.environment import *
from symbolicregression.metrics import *
from symbolicregression.model.embedders import conv_out_len
from symbolicregression.envs import ExpressionGeneratorArgs, ExpressionGenerator
from symbolicregression.envs import Node, NodeParseError


In [3]:
from parsers import default_params as params
from symbolicregression.envs import build_env

env = build_env(params)
env.set_rng(np.random.RandomState(0))

RandomState(MT19937) at 0x7F08F5896E40

In [8]:
for _ in range(10):
    expr, (x,y) = env.get_sample(n_observations=1, n_ops=15, max_n_vars=5)
    display(expr.to_sympy())

x_0*(0.32530241519407*x_4 - 0.915078427605696)*(x_0 + 1.48727035475266*x_3*x_4 - 0.292966715075799)

2.0*x_0 - x_1 + 6.11528112658438

-x_0**2*exp(2.3409742002214312*x_2)*cos(x_2) - x_0*(x_4 - (x_4 + 0.152963893022004)**2) - x_4 - 0.693998699721745

(-x_4*(0.0294432350765956*(x_0 + 0.207713594378862)**2 + 0.00786356950596983)/(-x_2 + 0.547683417949913/x_2) - 0.0978883529379355)**2

-x_1*(1.97906925231057*x_0 - 1.97906925231057*x_1 - 1.18586825335941) + cos(x_4)

-Abs(cos(0.3292053952573983*x_2 + 0.3292053952573983*sin((cos(x_3) - 0.7546519162351936)**4) - 0.2198271686473233)) - 0.327543741340636

1/(-x_0/(-1.44795089959083*x_1*(0.511144461007876 - x_0) + x_3 + 0.041960474314968) - x_4 + sin(x_4) + 0.0813638900195187)

sin(0.25477340329176307*Abs(x_0)*Abs(x_2) - 1.8139183199120317/Abs(x_3 + exp(x_2) + sqrt(sin(x_4))))**2

x_0 + 0.352339856898291*x_1 + (0.0449727270768282 - x_1)*(x_0 + (x_1 - 0.598124521632243)**2 + 0.397165440526039)**2 + 1.12599924007472

1.13845435376764*x_0/(-0.995864675965265*sin(x_1) - 1.44972457091326) - x_3 + 1.4443177275453

In [5]:
from parsers import default_params as params
params.min_vars=5
params.global_rank =1
dataloader = create_train_iterator(env, data_path="", params=params)

In [6]:
params=default_params
params.batch_size_eval = 1
params.eval_size = 200
iterator = create_test_iterator(env, "", params.srbench_path, params=params)


NameError: name 'default_params' is not defined

In [ ]:
import pickle
run_dir = Path("/checkpoint/pakamienny/new_symbolicregression/Feb17/tokens_per_batch_10000_lr_0.0002_accumulate_gradients_5_embedder_type_conv_use_emb_positional_embeddings_False/2023-02-17_08-15-20/")
 
#from parsers import default_params as params

params = pickle.load(open(run_dir / "params.pkl", "rb"))
params.reload_model = str(run_dir / "best-r2_train_median_synthetic.pth")

In [ ]:
from symbolicregression.model import build_modules

modules = build_modules(env, params)
embedder, encoder, decoder = (modules["embedder_module"], modules["encoder_module"], modules["decoder_module"])

output_word2id = modules["output_word2id"]
output_id2word = modules["output_id2word"]
output_tokenizer = modules["output_tokenizer"]


In [ ]:
env.get_sample(100, 15, 5)

In [ ]:
x.shape

In [ ]:
from symbolicregression.model.embedders import *
params.emb_hidden_size=256
params.emb_n_layers=2
pe = ConvEmbedder(modules["input_tokenizer"],  modules["input_word2id"], params)
_ = pe([(x,y)])

In [ ]:
modules["input_tokenizer"].encode(np.nan)

In [ ]:
embedder.eval()
encoder.eval()
decoder.eval()


def forward_greedy_model(x, y, optimize=False):
    x1, x1_len = embedder([(x[:params.n_max_observations], y[:params.n_max_observations])])
    encoded = encoder("fwd", x=x1, lengths=x1_len, causal=False).transpose(0, 1)
    generations, _ = decoder.generate(encoded, x1_len, sample_temperature=None, max_len=100)
    generations = generations.transpose(0, 1)

    ytildes = []
    for dataset_id, generation in zip(np.arange(len(datasets)), generations):
        words = [output_id2word[tok.item()] for tok in generation]
        while words[-1] == "<PAD>": words.pop(-1)
        assert words[0]=="<EOS>" and words[-1]=="<EOS>"
        try:
            decoded_expression: Node = output_tokenizer.decode(words[1:-1])
        except NodeParseError:
            continue
        if optimize:
            decoded_expression = run_torch_optim(decoded_expression, x, y, True)
        prefix = decoded_expression.prefix()
        ytilde = decoded_expression.evaluate(x)
        r2_train = stable_r2_score(y, ytilde)
        display(decoded_expression.to_sympy())
        ytildes.append(ytilde)
        print(decoded_expression, r2_train)
    plt.clf()
    plt.scatter(x[:params.n_max_observations], y[:params.n_max_observations], c="g")
    plt.scatter(x[params.n_max_observations:], y[params.n_max_observations:], c="r")

    for ytilde in ytildes:
        plt.scatter(x[:params.n_max_observations], ytilde[:params.n_max_observations], c="g", marker="x")
        plt.scatter(x[params.n_max_observations:], ytilde[params.n_max_observations:], c="r", marker="x")

    plt.show()

In [ ]:
expr, (x,y) = env.get_sample(params.n_max_observations+100, 10, 1) 
display(expr.to_sympy())



In [ ]:
import matplotlib.pyplot as plt

forward_greedy_model(x, y, False)



In [ ]:
forward_greedy_model(x, y, True)

In [ ]:
x1, x1_len = embedder(datasets)


In [ ]:
(x1==output_word2id["N11"]).sum()

In [ ]:
%%time

encoded = encoder("fwd", x=x1, lengths=x1_len, causal=False).transpose(0, 1)


In [ ]:
%%time

generations, _ = decoder.generate(encoded, x1_len, sample_temperature=None, max_len=50)


In [ ]:
generations

In [ ]:
df = pd.read_csv(run_dir / "eval_in-domain/epoch_25.csv", delimiter=";")#.drop(["Unnamed: 0"])
df["expression"] = df["expression"].apply(lambda x: Node.from_prefix(x))
df[["r2_train", "expression", "ground_truth"]].head(10)

In [ ]:
for expression in df["expression"]:
    print(Node.from_prefix(expression))